In [7]:
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2
from sklearn.svm import SVC,SVR
import os
import sys
from MFTreeSearchCV.MFTreeSearchCV import *
from mf.mf_func import *
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
from sklearn.linear_model import LogisticRegression

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
%ls MFTreeSearchCV/

converters.py   __init__.pyc  MFTreeFunction.py   MFTreeSearchCV.pyc
converters.pyc  MFHOO.py      MFTreeFunction.pyc  __pycache__/
__init__.py     MFHOO.pyc     MFTreeSearchCV.py


In [9]:
from sklearn.datasets import load_digits,load_boston,fetch_20newsgroups
data = load_boston()

In [10]:
newsgroups_train = fetch_20newsgroups(subset='all')
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(newsgroups_train.data)
labels = newsgroups_train.target
#features =features.todense()

In [11]:
X = features
y = labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape

(15076, 173762)

In [16]:
estimator = LogisticRegression() #base estimator
param_dict = {'C':{'range':[1e-5,1e5],'scale':'log','type':'real'},\
              'penalty':{'range':['l1','l2'],'scale':'linear','type':'cat'}} #parameter space
fidelity_range = [500,15076] # fidelity range, lowest fidelity uses 500 samples while the highest one uses 
#the whole dataset  
n_jobs = 3 # number of jobs
cv = 3 # cv level
fixed_params = {}
scoring = 'accuracy'

In [18]:
t1 = time.time()
estimator = estimator.fit(X_train,y_train)
t2 = time.time()
total_budget = 500 # total budget in seconds
print('Time without CV: ', t2 - t1)

('Time without CV: ', 13.485359907150269)


In [19]:
model = MFTreeSearchCV(estimator=estimator,param_dict=param_dict,scoring=scoring,\
                      fidelity_range=fidelity_range,unit_cost=None,\
                    cv=cv,  n_jobs = n_jobs,total_budget=total_budget,debug = True,fixed_params=fixed_params)

## running in debug mode will display certain outputs

In [20]:
m = model.fit(X_train,y_train)

Setting unit cost automatically as None was supplied
Unit Cost:  13.0087919235
Auto Init: 
C: 0.21970734557131216
nu: 0.21970734557131216
Budget Remaining: 481.698515857
Number of MFHOO Instances: 6
Budget per MFHOO Instance:67.2742940527
Running SOO number: 1 rho: 0.95 nu: 0.21970734557131216
Done!
Running SOO number: 2 rho: 0.940304092603 nu: 0.21970734557131216
Done!
Updating C
C: 0.307590283799837
nu_max: 0.307590283799837
Running SOO number: 3 rho: 0.925945462757 nu: 0.307590283799837
Done!
Updating C
C: 0.43062639731977176
nu_max: 0.43062639731977176
Running SOO number: 4 rho: 0.9025 nu: 0.43062639731977176
Done!
Updating C
C: 0.6028769562476805
nu_max: 0.6028769562476805
Running SOO number: 5 rho: 0.857375 nu: 0.6028769562476805
Done!
Updating C
C: 0.8440277387467526
nu_max: 0.8440277387467526
Running SOO number: 6 rho: 0.735091890625 nu: 0.8440277387467526
Done!


In [22]:
y_pred = m.predict(X_test)

In [23]:
accuracy_score(y_pred,y_test)

0.9259946949602123

In [24]:
m.best_params_

{'C': 5623.413251903499, 'penalty': 'l2'}

In [25]:
m.cv_results_

,params,score
0,"{u'penalty': u'l2', u'C': 23713.73705661653}",0.950717
1,"{u'penalty': u'l2', u'C': 74.98942093324557}",0.950448
2,"{u'penalty': u'l2', u'C': 5623.413251903499}",0.949528
3,"{u'penalty': u'l2', u'C': 5623.413251903499}",0.951048
4,"{u'penalty': u'l2', u'C': 5623.413251903499}",0.947862
5,"{u'penalty': u'l2', u'C': 5623.413251903499}",0.949723
